In [162]:
import pandas as pd
import json

from smart_open import smart_open
%matplotlib inline
import itertools
import numpy as np

In [10]:
def read_data():
    with smart_open('./data/dataset.jsonl.gz') as fh:
        dataset = [
            json.loads(line.decode('utf-8'))
            for line in fh
        ]
    return pd.DataFrame(dataset)

In [11]:
df = read_data()

In [33]:
df.shape

(101781, 9)

In [45]:
df.PRODUCT_ID.nunique()

71349

In [62]:
df[['BRAND','NAME']].sample(10)

,BRAND,NAME
53060,Levis Ambiance,Levis muurverf 'Ambiance' satijn artisjok 5L
65554,None,"Alabastine voorstrijk vlekken 2,5L"
67703,Decor,Decor split 'carrara' 20 kg
93913,None,Briloner verstelbare inbouwspot Attach dimbaar...
50837,ATTEMA,ATTEMA Tuit enkel 19 mm voor inbouwdoos U40/U5...
25173,Sencys,Sencys schietlood 3 m
8828,Duraline,Duraline wandkubus rond zwart 60x30cm
27990,Hamstra,HAMSTRA Horprofiel wit 150 cm
100612,Stanley,Stanley Interlock uitschuifmes
48143,Perfax,Perfax behangafsteker


In [55]:
df.SHOP.nunique()

5

In [68]:
df.head()

,BRAND,CONTENTS,DALTIX_ID,DESCRIPTION,DISPLAY_URL,NAME,PRODUCT_ID,SHOP,SPECIFICATIONS
0,Kärcher,[],43823f3f6826dcfd1f14b5898d742a1ead54937001f980...,De velgenreiniger 'Car & Bike' van Kärcher is ...,https://www.plan-it.be/nl/tuin-buitenleven/aut...,"Kärcher velgenreiniger 'Car & Bike' 0,5 L",5305104,planit,"{'Inhoud': '""500.0 ml""', 'Toepassing': '""Reini..."
1,Levi's,[],2258f16fa7ff7aab35c4534ac645017637d072f2fee9cc...,De matte muurverf Originals potloodgrijs van L...,https://www.bol.com/nl/p/muurverf-originals-ma...,"Muurverf Originals mat 2,5l potloodgrijs",9200000095428359,bol,None
2,Rapid,[],69ec8d4f776200897422a4b8f93d3416a36781fe110d75...,Vlakdraad nieten. Oorspronkelijk ontworpen voo...,https://www.bol.com/nl/p/rapid-nieten-no-970-8...,Rapid Nieten - No 970 - 8 mm,9200000010411327,bol,None
3,Differnz,[],f03ef881a1d1934c62b9db6b4403a59ae34de96148f362...,Toiletmeubel Fabulous 40 is een van de meest v...,https://www.bol.com/nl/p/differnz-fabulous-fon...,Differnz Fabulous Fonteinmeubel - 40 cm - Wit,9200000051330319,bol,None
4,Pickup,[],aaff0aa6db24814ad25d5cb410ded08361ab32bcb953e1...,"Pickup pictogram is zelfklevend, weerbestendig...",https://www.plan-it.be/nl/verf-laminaat-decora...,"Pickup pictogram P621 ""Heren""",1409779,planit,"{'Formaat': '""10 x 10cm""', 'Gebruik': '""Voor b..."


In [85]:
matches = [] 
for name, group in df.groupby('PRODUCT_ID'):
    matches += list(itertools.combinations(list(group['DALTIX_ID']), 2))

In [94]:
df = df.set_index('DALTIX_ID')

In [117]:
df['BRAND'] = df['BRAND'].fillna(df['NAME'].str.split(' ').str[0])

In [118]:
df_pairs = pd.DataFrame(matches,columns=['daltix_id_1','daltix_id_2'])

In [119]:
df_pairs['brand_id_1'] = df_pairs.daltix_id_1.apply(lambda x: df.loc[x,'BRAND'])
df_pairs['brand_id_2'] = df_pairs.daltix_id_2.apply(lambda x: df.loc[x,'BRAND'])

In [120]:
df_pairs['brand_id_1'] = df_pairs.brand_id_1.str.lower()

In [121]:
df_pairs['brand_id_2'] = df_pairs.brand_id_2.str.lower()

In [122]:
df_pairs['filter_brand'] = (df_pairs.brand_id_1 == df_pairs.brand_id_2) | pd.isnull(df_pairs.brand_id_1) | pd.isnull(df_pairs.brand_id_2)

In [123]:
first_submission = df_pairs[df_pairs.filter_brand == True]

In [127]:
first_submission[['daltix_id_1','daltix_id_2']].to_csv('firstsubmission.csv',index=False)

In [126]:
df_pairs['filter_brand'].value_counts()

True     27213
False     3238
Name: filter_brand, dtype: int64

In [128]:
len(df)

101781

In [137]:
df.PRODUCT_ID.value_counts()

5096960             3
5021003             3
5531371             3
6027021             3
5230263             3
5608565             3
5075309             3
6336913             3
5553352             3
5236894             3
5070057             3
5553354             3
5096967             3
5139636             3
5608507             3
5075339             3
5553353             3
5096977             3
5599178             3
9200000010648613    2
9200000058277974    2
3443615             2
9200000023061183    2
1994786             2
5227924             2
5541281             2
9200000050599785    2
9200000010402397    2
9200000033415786    2
9200000086678074    2
                   ..
9200000021232782    1
861219              1
584208              1
506368              1
875165              1
792402              1
847285              1
5633463             1
5024796             1
1111                1
7117900             1
5019532             1
2101266             1
5647823             1
175015    

In [144]:
df[['BRAND','PRODUCT_ID','SHOP','NAME']][df.PRODUCT_ID == "5531371"]

,BRAND,PRODUCT_ID,SHOP,NAME
DALTIX_ID,,,,
61d9c04542ac6040979e35b3961d1fb7ff09d2a151e03e514587f90e13bf8180,KOMAR,5531371,hornbach,KOMAR Fotobehang papier Rose Garden 368x254 cm
7ba0d49068ace47f2adbcc3749f5634251c7a412078a429b250322490b0d4975,Aurlane,5531371,planit,Aurlane douchecabine 'Pepper Mint' 85 x 85 cm
f17d207b54c7decb6af1e774a6d903bd4b6e899e884343052fb1f26874bc50bf,Douchecabine,5531371,praxis,Douchecabine Pepper Mint 230x85x85cm


In [155]:
df = read_data()

In [152]:
df[['NAME','BRAND']].sample(10)

,NAME,BRAND
39282,HG Schoendeo 250ml,HG
52404,FERM AGM1087 Haakse slijper 850W - 125mm,None
39009,GAH Alberts platprofiel staal warmgewalst 1 m ...,None
73743,Sencys universele schroeven 'Torx' staal gegal...,Sencys
28543,Pickup Plakletter J 90mm zwart mat,Pickup
65483,Domasol Manuele zonneluifel F10 350x300 cm ora...,Domasol
26151,3M SandBlaster schuurpapier op rol korrel 80 g...,None
1954,Busters veiligheidsbril 'Harrier I' kunststof,Busters
6868,Sencys afdekkap voor schroeven Pozi PZD 2 50 s...,None
98104,Tiger Melbourne Zeepdispenser - RVS Geborsteld,Tiger


In [156]:
df['NAME'] = df.NAME.str.lower()
df['BRAND'] = df.BRAND.str.lower()

In [158]:
df['first_word_name']  = df.NAME.str.split(' ').str[0]

In [160]:
df_2 = df[~pd.isnull(df.BRAND)]

In [165]:
np.sum(df_2.BRAND == df_2.first_word_name)/len(df_2) * 100

81.9750164365549

In [166]:
df.isnull().sum()

BRAND              25731
CONTENTS               0
DALTIX_ID              0
DESCRIPTION         3659
DISPLAY_URL            0
NAME                   0
PRODUCT_ID             0
SHOP                   0
SPECIFICATIONS     35060
first_word_name        0
dtype: int64

In [167]:
len(df_2)

76050

In [168]:
df.SHOP.nunique()

5